![imagenes](logo.png)

# SIUBA

El producto interno bruto (PIB) es una medida monetaria del valor de mercado de todos los bienes y servicios finales producidos y vendidos (no revendidos) en un período de tiempo específico por países.

El dataframe ``gapminder`` presenta información del PIB de varios países desde el año 1952 hasta el 2007 por periodos de 5 años.

Su informacion viene dada por las columnas:

* ``country``: país
* ``continent``: continente
* ``year``: año
* ``lifeExp``: esperanza de vida
* ``population``: población
* ``gdpPercap`` PIB per cápita

In [ ]:
## Mandamos a llamar nuestras bibliotecas principales

import pandas as pd
import numpy as np
from siuba import *
from siuba.dply.vector import *

In [ ]:
## Leemos el archivo gapminder.csv
gapminder = pd.read_csv("gapminder.csv")

## Muestra las primeras filas de gapminder

In [ ]:
gapminder >> head(15) # gapminder.head() 

## Piensa antes de trabajar los datos

Una variable, o columna, de tipo categórica es aquella que indica si un renglón pertenece a alguna clase. Por ejemplo, en 

nombre|sexo|estatura
:-:|:-:|:-:
Luis|masc|1.70|
María|feme|1.65
Marco|masc|1.83
Helena|feme|1.61
Luisa|feme|1.67

la columna ``sexo`` nos indica si el renglón corresponde a la clase ``masc`` o a la clase ``feme``. A los diferentes valores que puede tomar una columna que utilizamos para clasificar se les llama *niveles* o *factores*.

Si tienes la necesidad de almacenar un pequeño fragmento de sus datos, detente y pregúntate **¿Quiero crear mini conjuntos de datos para cada nivel de alguna clase (o combinación única de varias columnas categóricas) para calcular o graficar algo?**

En **caso afirmativo**, usa técnicas de agregación de datos adecuadas. **No dividas los datos en subconjuntos**. O solo divide los datos en subconjuntos como una medida temporal mientras desarrollas tu código para computar o visualizar estos subconjuntos de datos.

Si **NO**, entonces tal vez realmente necesitas almacenar una copia de un subconjunto de los datos.

Toma en cuenta que las copias y extractos de tus datos saturan tu espacio de trabajo, invitan a cometer errores y generan confusión general. 

La realidad también puede estar en algún punto intermedio. Encontrarás que los flujos de trabajo que se presentan a continuación pueden ayudarte a lograr tus objetivos con una creación mínima de objetos intermedios temporales.

## filter() para crear subconjuntos de datos por filas.

``filter()`` toma expresiones lógicas y devuelve las filas para las que todos esas expresiones son verdaderas.

**Objetivos:**

* Encuentra las observaciones donde la esperanza de vida sea inferior a 29
* Encuentra las observaciones de Ruanda después del año 1979 (Ruanda en inglés es Rwanda)
* Encuentra las observaciones tanto de Ruanda como de Afganistán (Afganistán en inglés es Afghanistan)

In [ ]:
gapminder >> filter(_.lifeExp < 29)  # gapminder[gapminder["lifeExp"] < 29]

In [ ]:
gapminder >> filter(_.country == "Rwanda", _.year > 1979) 
# gapminder[(gapminder["country"] == "Rwanda") & (gapminder["year"] > 1979)]

In [ ]:
gapminder >> filter((_.country == "Rwanda") | (_.country == "Afghanistan"))
# gapminder[(gapminder["country"] == "Rwanda") | (gapminder["country"] == "Afghanistan")]

In [ ]:
gapminder[(gapminder["country"] == "Rwanda") | (gapminder["country"] == "Afghanistan")]

### El método ``.isin()`` de pandas

Alternativamente, podemos emplear el método ``.isin()`` de pandas:

``columna.isin(<<lista>>)``


In [ ]:
gapminder >> filter(_.country.isin(["Rwanda","Afghanistan"]))
# gapminder[gapminder["country"].isin(["Rwanda","Afghanistan"])]

## select() para crear subconjuntos de datos en variables o columnas.

Utiliza ``select()`` para crear un subconjunto de los datos de las variables.

**Objetivos:**

* Selecciona las variables ``lifeExp`` y ``year``. Nota que el orden en que estas columnas aparecen en ``gapminder`` es primero ``year`` y luego ``lifeExp``
* Obtener las variables ``year`` y ``lifeEx`` para Camboya (Camboya en inglés es Cambodia)

In [ ]:
gapminder >> select(_.lifeExp, _.year)

In [ ]:
gapminder >> filter(_.country == "Cambodia") >> select(_.year,_.lifeExp) 

# gapminder[gapminder["country"] == Cambodia][["year","lifeExp"]]

Nota que el orden de los verbos altera el resultado: si primero seleccionamos ``year`` y ``lifeExp``, el dataframe resultante no tendrá columna ``country``,  de modo que no podremos recuperar Camboya

In [ ]:
gapminder >> select(_.year,_.lifeExp) >> filter(_.country == "Cambodia") 

In [ ]:
gapminder %>% select(year,lifeExp) %>% filter(country == "Cambodia")

### Los métodos avanzados de selección dentro de pandas

Considere el siguiente dataframe

In [ ]:
np.random.seed(2021)
mi_dataframe = {"nombre":["Luis","María","Helena"],
                "Sexo":["masc","feme","feme"],
                "Materia_matemáticas":np.random.randint(0,11,3),
                "Materia_español":np.random.randint(0,11,3),
                "Materia_inglés":np.random.randint(0,11,3),
                "Materia_computación":np.random.randint(0,11,3),
                "Evaluación_trimestral":np.random.randint(0,11,3),
                "Evaluación_general":np.random.randint(0,11,3),
                "Trimestral_aprovechamiento":["No","R","No"],
                "General_aprovechamiento":["No","No","Sí"]}

mi_dataframe = pd.DataFrame(mi_dataframe)
mi_dataframe

Hay ocasiones en las cuales debemos seleccionar muchas columnas de un dataframe. Pueden ser tantas, que escribir una por una dentro de un select puede llegar a ser tedioso.

**Objetivos:**

* Mostrar únicamente el nombre y las calificaciones de las materias de las mujeres.
* Mostrar únicamente el nombre y las calificaciones de las materias y las evaluaciones de las mujeres.
* Mostrar únicamente el nombre, las evaluaciones y los aprovechamientos.

In [ ]:
mi_dataframe >> filter(_.Sexo == "feme") >> select(_.nombre, _.startswith("Materia"))

In [ ]:
mi_dataframe >> filter(_.Sexo == "feme") >> select(_.nombre, _.startswith(("Evaluación","Materia")))

In [ ]:
mi_dataframe >> select(_.nombre, _.startswith("Evaluación"),_.endswith("aprovechamiento"))

--------
--------
**Tarea.**

Intenta crear la tabla anterior utilizando únicamente ``pandas`` y ``startswith``.

--------
--------

En [esta página](https://siuba.readthedocs.io/en/latest/api_table_core/03_select.html) encontrarás varios métodos de selección en siuba.

## mutate() para agregar nuevas variables

Imagina que quisiéramos recuperar el PIB de cada país. Después de todo, los datos de ``gapminder`` tienen una variable para población (``pop``) y PIB per cápita (``gdpPercap``). 

``mutate()`` es una función que define e inserta nuevas variables en un dataframe. Puedes hacer referencia a las variables existentes por su nombre.

In [ ]:
gapminder >> mutate(gdp = _.gdpPercap * _.population)

In [ ]:
## Quitando la notación científica en todos los dataframes de la sesión actual
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
gapminder >> mutate(gdp = _.gdpPercap * _.population)

In [ ]:
## Eliminando el formato de dos decimales que afecta globalmente
pd.reset_option('display.float_format')

In [ ]:
## Quitando la notación científica únicamente para la columna de interés

gapminder >> mutate(gdp = (_.gdpPercap * _.population).apply(lambda x: '%.2f' % x))

In [ ]:
gapminder >> filter(_.country == "Venezuela")

Esas cifras del PIB son casi inútilmente grandes y abstractas: "Una cosa que me molesta son los números grandes que se presentan sin contexto… Si añado un cero a este número, ¿la oración que lo contiene significaría algo diferente para mí? Si la respuesta es 'no', tal vez el número no tiene por qué estar en la oración en primer lugar".

Tal vez sería más significativo para los consumidores de tus tablas y cifras ceñirse al PIB per cápita. Pero, ¿y si reportara el PIB per cápita *relativo con algún país de referencia*? Ya que México es mi hogar, me quedo con eso. Es decir, cuánto representa el PIB per cápita de cada país respecto del de México. 

Necesito crear una nueva variable que se divida gdpPercap por el de PIB per cápita de México, cuidando que siempre divida dos números que pertenezcan al mismo año.

In [ ]:
gapminder >> filter(_.country == "Mexico")

In [ ]:
142*(gapminder >> filter(_.country == "Mexico"))["gdpPercap"].tolist()

In [ ]:
gdpPercap_Mex = (gapminder >> filter(_.country == "Mexico"))["gdpPercap"].tolist()

In [ ]:
gapminder.shape[0]/12

In [ ]:
total_paises = int(gapminder.shape[0]/12)

In [ ]:
total_paises*gdpPercap_Mex

In [ ]:
mi_gapminder = (gapminder >> 
    mutate(gdpPercap_respect_Mex = _.gdpPercap*100/(total_paises*gdpPercap_Mex))
    )

In [ ]:
mi_gapminder

In [ ]:
mi_gapminder >> filter(_.country == "Mexico")

In [ ]:
mi_gapminder >> filter(_.gdpPercap_respect_Mex >= 98,_.gdpPercap_respect_Mex <= 103)

**Objetivos:**

* Mostrar los renglones de los países que estuvieron, en algún año, mejor que México
* Mostrar los renglones de los países americanos que estuvieron, en algún año, mejor que México
* Mostrar cómo evolucionó la economía de Venezuela respecto de la de México 

In [ ]:
mi_gapminder >> filter(_.gdpPercap_respect_Mex > 100)

In [ ]:
mi_gapminder >> filter(_.gdpPercap_respect_Mex > 100, _.continent == "Americas")

In [ ]:
mi_gapminder >> filter(_.country == "Venezuela")

## arrange() para ordenar datos por filas de una manera basada en principios

``arrange()`` reordena las filas en un dataframe.

**Objetivo:**

* Ordenar los datos por año y luego por país, en lugar de por país y luego por año.
* ¿O tal vez solo desea los datos de 2007, ordenados según la esperanza de vida?
* Ah, ¿te gustaría clasificar la esperanza de vida en orden descendente ?

In [ ]:
gapminder >> arrange(_.year,_.country)

In [ ]:
gapminder >> filter(_.year == 2007) >> arrange(_.lifeExp)

In [ ]:
gapminder >> arrange(-_.lifeExp)

Te aconsejo que tus análisis NUNCA se basen en filas o variables en un orden específico. Pero una vez que estés preparando tablas para ojos humanos, es imperativo que tomes el control del orden de las filas.

## rename() para renombrar variables

¡Cambiemos el nombre de algunas variables!

**Objetivo:**

*  Renombrar a las columnas ``country`` y ``gdpPercap`` por su nombre en español

In [ ]:
gapminder >> rename(pais = _.country, pib_percap = _.gdpPercap)

In [ ]:
gapminder >> distinct(_.year) 

### select() también puede utilizarse para cambiar los nombres de las columnas

Has visto el uso simple de ``select()``. Hay otro truco que te puede gustar:

* ``select()`` puede cambiar el nombre de las variables que solicita mantener. Puedes renombrar una columna en específico utilizando el operador ``==`` de la siguiente manera:

``_.nuevo_nombre == _.viejo_nombre``

In [ ]:
(gapminder >>
    filter(_.country == "Burundi", _.year > 1996) >>
    select(_.yr == _.year, _.lifeExp, _.gdpPercap)
    ) 

## group_by() es un arma poderosa

Todos hemos tenido colaboradores a los cuales les encanta hacer preguntas aparentemente simples como "¿qué país experimentó la mayor caída en la esperanza de vida en 5 años?". De hecho, esa es una pregunta totalmente natural. Pero si no estás utilizando un lenguaje que procese datos, es una pregunta increíblemente molesta de responder.

**siuba** ofrece herramientas poderosas para resolver este tipo de problema.

* ``group_by()`` agrega estructura adicional a su conjunto de datos (información de agrupación) que sienta las bases para los cálculos dentro de los grupos.

* ``summarize()`` toma un conjunto de datos con $n$ observaciones, calcula los resúmenes solicitados y devuelve un conjunto de datos con 1 observación.

* ``mutate()`` y ``summarize()`` distinguen entre grupos.

### Contando cosas

Comencemos con el conteo simple. ¿Cuántas observaciones tenemos por continente?

In [ ]:
gapminder >> group_by(_.continent) >> count()

Alternativamente, podemos utilizar varias funciones incluídas en ``siuba.dply.vector``. Puedes ver muchas de ellas en https://siuba.readthedocs.io/en/latest/api_extra/vector.html

En particular, su función de conteo es ``n()``

In [ ]:
gapminder >> group_by(_.continent) >> summarize(total = n(_))

¿Qué pasaría si quisiéramos contar la cantidad de países únicos para cada continente? Puedes calcular múltiples resúmenes dentro de summarize(). 

Utiliza  ``n_distinct()`` para contar el número de países distintos dentro de cada continente.

In [ ]:
gapminder >> group_by(_.continent) >> summarize(total = n(_),paises_distintos = n_distinct(_.country))

## Resúmenes más generales

Las funciones que aplicará ``summarize()`` incluyen resúmenes estadísticos clásicos. En particular, cualquiera de las que encontramos en la sección de funciones estadísticas de **numpy**

In [ ]:
gapminder >> group_by(_.continent) >> summarize(mean_lifeExp = _.lifeExp.mean())

In [ ]:
mi_gap = gapminder.copy()

In [ ]:
mi_gap["lifeExp"][0] = np.nan

In [ ]:
mi_gap >> group_by(_.continent) >> summarize(mean_lifeExp = _.lifeExp.mean())

Centrémonos sólo en Asia. ¿Cuáles son las esperanzas de vida mínima y máxima vistas por año?

In [ ]:
(gapminder >> 
    filter(_.continent == "Asia") >> 
    group_by(_.year) >> 
    summarize(esp_min = _.lifeExp.min(), esp_max=_.lifeExp.max())
    )

Por supuesto, sería mucho más interesante ver qué país contribuyó con estas observaciones extremas. ¿El mínimo (máximo) viene siempre del mismo país? Abordaremos eso en breve.

## Mutación agrupada

A veces no deseas colapsar  las $n$ filas de cada grupo en una sola fila, sino mantener tus grupos, pero calculando nuevas columnas dentro de ellos.

Hagamos una nueva variable que sean los años de esperanza de vida ganados (perdidos) en relación con 1952, para cada país individual. Por ejemplo, para el caso de México, queremos la siguiente tabla:

country|continent|year|lifeExp|population|gdpPercap|gain_lifeExp
:-:|:-:|:-:|:-:|:-:|:-:|:-:|
Mexico|Americas|1952|50.789|30144317|3478.125529|0
Mexico|Americas|1957|55.19|35015548|4131.546641|4.401
Mexico|Americas|1962|58.299|41121485|4581.609385|7.51
Mexico|Americas|1967|60.11|47995559|5754.733883|9.321
Mexico|Americas|1972|62.361|55984294|6809.40669|11.572
Mexico|Americas|1977|65.032|63759976|7674.929108|14.243
Mexico|Americas|1982|67.405|71640904|9611.147541|16.616
Mexico|Americas|1987|69.498|80122492|8688.156003|18.709
Mexico|Americas|1992|71.455|88111030|9472.384295|20.666
Mexico|Americas|1997|73.67|95895146|9767.29753|22.881
Mexico|Americas|2002|74.902|102479927|10742.44053|24.113
Mexico|Americas|2007|76.195|108700891|11977.57496|25.406


Agrupamos por país y usamos ``mutate()`` para hacer una nueva variable. La función ``first()`` extrae el primer valor de un vector. 

Observa que ``first()`` está operando sobre el vector de esperanza de vida dentro de cada grupo de países.

In [ ]:
gapminder >> group_by(_.country) >> mutate(gain_lifeExp = _.lifeExp-first(_.lifeExp))

Si ya no harás ninguna operación extra en el dataframe anterior que involucre grupos, conviene cerrar el agrupamiento con el verbo ``ungroup()``

In [ ]:
gapminder >> group_by(_.country) >> mutate(gain_lifeExp = _.lifeExp-first(_.lifeExp)) >> ungroup()

Por el contrario, si todavía vas a trabajar con los grupos, recuerda que tanto ``mutate()`` como ``summarize()`` distinguen los grupos.

In [ ]:
(gapminder >>
    group_by(_.country) >> 
    mutate(gain_lifeExp = _.lifeExp-first(_.lifeExp)) >>
    summarize(mean_gain_lifeExp = _.gain_lifeExp[1:].mean())
    )

## Top_n

Con ``top_n`` podemos pedir que nos devuelva los mejores ``n`` renglones respecto de una columna. 

**Objetivos.**

* Para cada año, mostrar el año, el país y la esperanza de vida que tuvieron los tres países con menor esperanza de vida en Asia.

* Para cada año, mostrar el año, el país y la esperanza de vida que tuvieron el país con mayor y menor esperanza de vida en Asia.

In [ ]:
(gapminder >>
  filter(_.continent == "Asia") >>
  select(_.year, _.country, _.lifeExp) >>
  group_by(_.year) >>
  top_n(3, wt = -_.lifeExp) >>
  arrange(_.year, _.lifeExp) >>
  mutate(lugar = así c[1,2,3]) >>
  ungroup()
)

In [ ]:
(gapminder >>
    filter(_.continent == "Asia") >>
    select(_.year,_.country,_.lifeExp) >>
    group_by(_.year) >>
    filter((_.lifeExp == _.lifeExp.min()) | (_.lifeExp == _.lifeExp.max())) >>
    arrange(_.year,_.lifeExp) >>
    mutate(type = ["minimo","maximo"]) >>
    ungroup()
    )

Estamos ahora en condiciones de responder *¿qué país experimentó la mayor caída en la esperanza de vida en 5 años?*

In [ ]:
(gapminder >>
  select(_.country, _.year, _.continent, _.lifeExp) >>
  group_by(_.country) >>
  ## dentro de country, calcular (lifeExp in year i) - (lifeExp in year i - 1)
  ## Si el resultado es positivo, entonces lifeExp aumentó; si el resultado es negativo, entonces disminuyó
  mutate(le_delta = _.lifeExp - lag(_.lifeExp))   >> 
  ## dentro de country, nos quedamos con la fila con menor le_delta
  filter(_.le_delta == _.le_delta.min()) >>
  ungroup() >>
  ## agrupamos por continente
  group_by(_.continent) >>
  ## dentro de continente, seleccionamos el renglón con le_delta mas bajo
  top_n(-1,wt = _.le_delta) >>
  ungroup() >>
  arrange(_.le_delta)
)

## Joins

Supongamos que tenemos los siguientes dataframes:

In [ ]:
df1 = pd.DataFrame({'id': [1,2], 'x': ['a', 'b']})
df2 = pd.DataFrame({'id': [2,2,3], 'y': ['l', 'm', 'n']})


In [ ]:
df1

In [ ]:
df2

Como puede observarse, ambos tienen en común la información ``id``. Cabe preguntarse *¿podré llevarme la informació de una tabla a la otra mediante esa columna?* La respuesta nos la proporcionan las funciones ``join``.

**Objetivos:**

* Mostrar la información **completa** que tienen en común los dos dataframes
* Llevar la información desde el dataframe ``df2`` hacia el ``df1``

In [ ]:
df1 >> inner_join(_, df2, on = "id")

In [ ]:
df1 >> left_join(_, df2, on = "id")

In [ ]:
df2 >> left_join(_, df1, on = "id")